In [1]:
import spotipy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from spotipy.oauth2 import SpotifyOAuth
from main_functions import *

#Librerías de IA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

plt.rcParams['font.family'] = 'DejaVu Sans'

print("¡Entorno listo para Spotify Wrapped!")

¡Entorno listo para Spotify Wrapped!


In [2]:
sp = authenticate_spotify()

✅ Caché eliminada. Inicia sesión con una nueva cuenta.
¡Autenticado como: rubmuiper!


IA QUE PREDICE SI TE GUSTARÁ UNA CANCIÓN O NO

In [6]:
top_tracks = get_top_tracks(sp, limit=50)  # Obtiene las 50 canciones más escuchadas

track_features = []
for track in top_tracks['items']:
    track_info = sp.audio_features(track['id'])[0]  # Obtiene características acústicas de la canción
    track_data = {
        'name': track['name'],
        'artist': track['artists'][0]['name'],
        'popularity': track['popularity'],
        'tempo': track_info['tempo'],
        'energy': track_info['energy'],
        'danceability': track_info['danceability'],
        'valence': track_info['valence'],
        'loudness': track_info['loudness'],
        'release_year': int(track['album']['release_date'][:4]),
    }
    track_features.append(track_data)

# Crear un DataFrame con las características de las canciones más escuchadas
track_df = pd.DataFrame(track_features)

HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=6V78ktWCGcJIIB5DPJBoPn with Params: {} returned 403 due to None


SpotifyException: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=6V78ktWCGcJIIB5DPJBoPn:
 None, reason: None

In [ ]:
# Obtiene características acústicas de una canción específica
song_name = "Shape of You"  # Ejemplo de nombre de canción
result = sp.search(song_name, limit=1)  # Busca la canción por nombre

track_info = result['tracks']['items'][0]
track_id = track_info['id']
audio_features = sp.audio_features(track_id)[0]

new_song_features = {
    'tempo': audio_features['tempo'],
    'energy': audio_features['energy'],
    'danceability': audio_features['danceability'],
    'valence': audio_features['valence'],
    'loudness': audio_features['loudness'],
    'release_year': int(track_info['album']['release_date'][:4]),
}

# Convertir a DataFrame y predecir
new_song_df = pd.DataFrame([new_song_features])
predicted_liked = model.predict(new_song_df)

# Mostrar probabilidad de que te guste
predicted_probability = model.predict_proba(new_song_df)[:, 1]
print(f"Probabilidad de que te guste '{song_name}': {predicted_probability[0] * 100:.2f}%")


In [4]:
# Procesa los datos: crea la columna 'liked', convierte géneros a variables dummy, 
# realiza Label Encoding para 'artist' y elimina columnas no necesarias.

track_df['liked'] = track_df['popularity'].apply(lambda x: 1 if x > 70 else 0)

if 'genres' in track_df.columns:
    track_df = track_df.join(track_df['genres'].str.join('|').str.get_dummies())
    track_df.drop('genres', axis=1, inplace=True)

label_encoder = LabelEncoder()
track_df['artist'] = label_encoder.fit_transform(track_df['artist'])

track_df.drop('name', axis=1, inplace=True)

In [5]:
# Entrena un modelo de Random Forest para predecir si te gusta una canción y calcula la precisión.

X = track_df.drop('liked', axis=1)
y = track_df['liked']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Precisión del modelo: {accuracy * 100:.2f}%")

probabilidad = model.predict_proba(X_test)

probabilidad_te_guste = probabilidad[:, 1]
print(f"Probabilidad de que te guste la canción: {probabilidad_te_guste[0] * 100:.2f}%")


Precisión del modelo: 100.00%
Probabilidad de que te guste la canción: 4.00%
